In [7]:
!pip install transformers peft trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-fr

In [8]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00


In [9]:
!pip install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00


In [10]:
import os
import torch
import gc
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,AutoTokenizer
from transformers import TrainingArguments,pipeline
from peft import LoraConfig, PeftModel, get_peft_config
from trl import SFTTrainer

In [11]:
import warnings

warnings.filterwarnings("ignore")

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
device_map = "auto"

In [14]:
df = pd.read_csv("/content/drive/MyDrive/SKRIPSIWET/Fine-Tuning-LLMs/data/medquad.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SKRIPSIWET/Fine-Tuning-LLMs/data/medquad.csv'

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
data = Dataset.from_pandas(pd.DataFrame(data=df))

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
#"daryl149/llama-2-7b-chat-hf"
#model_name = "meta-llama/Llama-2-7b-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map=device_map,
    )
model.config.pretraining_tp = 1
torch.cuda.empty_cache()

In [ ]:
LORA_ALPHA = 16
LORA_DROPOUT = 0.2
LORA_R = 64

In [ ]:
peft_config = LoraConfig(
        lora_alpha= LORA_ALPHA,
        lora_dropout= LORA_DROPOUT,
        r= LORA_R,
        bias="none",
        task_type="CAUSAL_LM",
    )

In [ ]:
LEARNING_RATE = 1e-4
NUM_EPOCHS = 10
BATCH_SIZE = 16
WEIGHT_DECAY = 0.001
MAX_GRAD_NORM = 0.3
gradient_accumulation_steps = 16
STEPS = 1
OPTIM = "adam"
MAX_STEPS = 10

In [ ]:
OUTPUT_DIR = "./results"

In [ ]:
training_args = TrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps= gradient_accumulation_steps,
    learning_rate= LEARNING_RATE,
    logging_steps= STEPS,
    num_train_epochs= NUM_EPOCHS,
    max_steps= MAX_STEPS,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="question",
    max_seq_length=500,
    tokenizer=tokenizer,
    args=training_args,
)

In [ ]:
trainer.train()

In [ ]:
training_args = TrainingArguments(
    output_dir="finetuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Decrease batch size
    save_steps=10_000,
    save_total_limit=2,
    gradient_accumulation_steps=4,  # Gradient accumulation
    gradient_checkpointing=True,  # Gradient checkpointing
    fp16=True,  # Mixed precision training
)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple model, loss function, and optimizer
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(3 * 224 * 224, 2)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Assuming you have a model, loss function, and optimizer defined somewhere
model = SimpleModel().cuda()  # Move the model to the GPU
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Define the trace handler function
def trace_handler(prof):
    # Save the trace to a file
    prof.export_chrome_trace("trace.json")

# Initialize the profiler context
with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(wait=0, warmup=0, active=6, repeat=1),
    record_shapes=True,
    profile_memory=True,
    with_stack=True,
    on_trace_ready=trace_handler,
) as prof:
    # Dummy data for illustration purposes
    original_batch_size = 10
    new_batch_size = 5  # Adjust based on available GPU memory
    inputs = torch.randn((new_batch_size, 3, 224, 224)).cuda().float()  # Change data type to Float
    labels = torch.randint(0, 2, (new_batch_size,)).cuda()

    # Run the PyTorch Model inside the profile context
    for _ in range(5):
        prof.step()
        with torch.profiler.record_function("## forward ##"):
            pred = model(inputs)

        with torch.profiler.record_function("## backward ##"):
            loss_fn(pred, labels).backward()

        with torch.profiler.record_function("## optimizer ##"):
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

        torch.cuda.empty_cache()  # Free up GPU memory

# Construct the memory timeline HTML plot
prof.export_memory_timeline("timeline.html", device="cuda:0")



In [16]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
torch.cuda.empty_cache()

In [ ]:
pipe = pipeline(task="text-generation",model=model,tokenizer=tokenizer,max_length=500)

In [ ]:
prompt = "What is Glaucoma?"

In [ ]:
template = f"""<s>[INST] <<SYS>>
You are a helpful, respectful and honest Medical assistant.
You always answer to the context and do not hallucinate. You only answer to the questions that are related to Medical and Healthcare.
If a question does not make any sense,Just answer I don't know, please don't share false information.
<</SYS>>
{prompt} [/INST]
"""

In [ ]:
result = pipe(template)

In [ ]:
response = result[0]['generated_text']
response

In [ ]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the fine-tuned model and tokenizer
model_name = "your_model_name"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to generate a response
def generate_response(input_text):
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=100, num_beams=5, early_stopping=True)
    response = tokenizer.decode(outputs[0])
    return response

# Streamlit App
st.title("Fine-Tuned Llama Model")
input_text = st.text_input("Enter your text:", "Type here...")

if st.button("Generate Response"):
    if input_text:
        response = generate_response(input_text)
        st.write(response)
    else:
        st.write("Please enter text for generation.")